In [1]:

# conda remove --name SCENIC --all
# conda create --name SCENIC python=3.6.3 
# conda activate SCENIC
# python -m pip install pandas
# git clone https://github.com/aertslab/pycisTopic.git
# cd pycisTopic
# python -m pip install .
# python -m pip install --upgrade pip setuptools
# python -m pip install scanpy
# python -m pip install  pyranges
# python -m pip install requests --ignore-installed certifi
# python -m pip install ray
# python -m pip install tmtoolkit
# python -m pip install gensim
# python -m pip install loomxpy
# python -m pip install pyrle
# python -m pip install polars
# python -m pip  install typing-extensions --upgrade
# python -m pip install pyarrow
# python -m pip install loompy
# python -m pip install pyscenic
# python -m pip install pickle5
# python -m pip install ipykernel
#  python -m pip install pyBigWig
# install jupyter-lab
# mv pycisTopic/ pycisTopic_repo
#git clone https://github.com/aertslab/scenicplus
#cd scenicplus
#pip install .




import warnings
warnings.simplefilter(action='ignore')
import pandas as pd
import scanpy
import os
import requests
import pyranges as pr
import pycisTopic
from pycisTopic.pseudobulk_peak_calling import export_pseudobulk
from pycisTopic.iterative_peak_calling import *
import anndata
import pickle
import pyarrow 
import re
from pycisTopic.lda_models import *
from pycisTopic.cistopic_class import *
import scrublet as scr
from pycisTopic.lda_models import *
from pycisTopic.clust_vis import *



def CountFrequency(my_list):
 
    # Creating an empty dictionary
    freq = {}
    for items in my_list:
        freq[items] = my_list.count(items)
 
    for key, value in freq.items():
        print("% s : % d" % (key, value))
 

outDir="SCENIC_results_MARCH2024/"
tmpDir="/tmp/"
#ray.shutdown()

In [2]:
import pickle
infile = open(outDir + 'topic_binarization/binarized_topic_region.pkl', 'rb')
binarized_topic_region = pickle.load(infile)
infile.close()

import pickle
infile = open(outDir + 'DARs/DARs.pkl', 'rb')
DARs_dict = pickle.load(infile)
infile.close()

In [3]:
# Load data
## ATAC - cisTopic object
import pickle
infile = open(outDir + 'cisTopicObject_noDBL.pkl', 'rb')
cistopic_obj = pickle.load(infile)
infile.close()
## Precomputed imputed data
import pickle
infile = open( outDir + 'DARs/Imputed_accessibility.pkl', 'rb')
imputed_acc_obj = pickle.load(infile)
infile.close()
## RNA - Create Anndata
from loomxpy.loomxpy import SCopeLoom
from pycisTopic.loom import *
import itertools
import anndata




In [4]:
infile = open(outDir + 'topic_binarization/Topic_qc_metrics_annot.pkl', 'rb')
topic_qc_metrics = pickle.load(infile)
infile.close()
 
df = pd.DataFrame(topic_qc_metrics)
df.to_csv((outDir + 'topic_qc_metrics.csv'))

In [5]:
cell_data=scanpy.read_h5ad("sobj.merged.h5ad")
cluster='cluster_'
cell_df=cell_data.obs
barcodes_names=list(cell_df.index)
cell_df['barcodes_names']=barcodes_names
barcodes=[re.sub(".df_", "", x) for x in barcodes_names]
cell_df['barcodes']=barcodes
cell_df['cellnames']=cell_df['barcodes'] + '-' + cell_df['orig.ident']
cell_df['clusters']=cluster + cell_df['seurat_clusters'].astype(str)
cell_df.index = cell_df['cellnames']
cell_df.index
metadata=cell_df

In [6]:

# Fix names
metadata['barcode'] = [x.split('-')[0] for x in metadata.index.tolist()]

cell_data.index=cell_df.index 

In [7]:
type(cell_data)

anndata._core.anndata.AnnData

In [8]:
import pyranges as pr
from pycistarget.utils import *

region_sets = {}
#region_sets = {key: pr.PyRanges(region_names_to_coordinates(binarized_topic_region[key].index.tolist())) for key in binarized_topic_region.keys()}
region_sets['Topics'] = {key: pr.PyRanges(region_names_to_coordinates(binarized_topic_region[key].index.tolist())) for key in binarized_topic_region.keys()}
#region_sets['DARs'] = {re.sub('[^A-Za-z0-9]+', '_', key): pr.PyRanges(region_names_to_coordinates(DARs_dict[key].index.tolist())) for key in DARs_dict.keys()}


In [9]:
pwd

'/mnt/ceph/mbp/servers/bioinformatics-platform/home/lper0012/tasks/margo.montandon/Analysis/SCENIC_MARCH2024'

In [10]:

# in here run creating_database_for_cistarget.bash before the following steps
#Writing cisTarget regions vs motifs scores db: "Drerio_GRCz11_ensembl.motifs_vs_regions.scores.feather"
#Writing cisTarget motifs vs regions scores db: "Drerio_GRCz11_ensembl.regions_vs_motifs.scores.feather"
#Create rankings from "Drerio_GRCz11_ensembl.motifs_vs_regions.scores.feather" with random seed set to 555.
#Writing cisTarget motifs vs regions rankings db: "Drerio_GRCz11_ensembl.regions_vs_motifs.rankings.feather"


from pycistarget.motif_enrichment_cistarget import *

rankings_db = '../Drerio_GRCz11_ensembl.regions_vs_motifs.rankings.feather'
scores_db = '../Drerio_GRCz11_ensembl.regions_vs_motifs.scores.feather'
#motif_annotation = 'motifs-v10-zfish.tbl' # this file is generated with SCENIC_motifs_ort.ipynb
#motif_annotation = 'motifs-v10-zfish_updated.tbl' # this file is generated with SCENIC_motifs_ort.ipynb
motif_annotation = '../motifs-v10-zfish_updated_allspecies.tbl'
#ctx_db = cisTargetDatabase(rankings_db, region_sets)   

# Remove dbcorr motifs (only if using cisTarget v9)
#ctx_db.db_rankings = ctx_db.db_rankings[~ctx_db.db_rankings.index.str.contains("dbcorr")]
#ctx_db.db_rankings

#region_sets['DARs'] = {re.sub('[^A-Za-z0-9]+', '_', key): pr.PyRanges(region_names_to_coordinates(DARs_dict[key].index.tolist())) for key in DARs_dict.keys()}


In [20]:

import pybiomart as pbm
import pandas as pd

#run_pycistarget(
#    region_sets = region_sets,
#    species = 'homo_sapiens',
#    save_path = os.path.join(outDir, 'motifs'),
#    ctx_db_path = rankings_db,
#    dem_db_path = scores_db,
#    path_to_motif_annotations = motif_annotation,
#    run_without_promoters = True,
#    n_cpu = 8,
#    _temp_dir = os.path.join(tmpDir, 'ray_spill'),
#    annotation_version = 'v10nr_clust',
#    )
#dataset = pbm.Dataset(name='drerio_gene_ensembl',  host='http://asia.ensembl.org')
#annot = dataset.query(attributes=['chromosome_name', 'transcription_start_site', 'strand', 'external_gene_name', 'transcript_biotype'])
annot=pd.read_csv('annot1.csv')
annot

,Chromosome/scaffold name,Transcription start site (TSS),Strand,Gene name,Transcript type
0,24,20927202,1,fam162a,protein_coding
1,24,20927135,1,fam162a,protein_coding
2,17,50472985,-1,si:ch211-235i11.3,protein_coding
3,17,50472864,-1,si:ch211-235i11.3,protein_coding
4,1,49266886,1,caly,protein_coding
...,...,...,...,...,...
61625,11,43070883,1,CABZ01080006.1,lincRNA
61626,11,43643158,1,CABZ01073015.1,lincRNA
61627,11,44120354,-1,FO704721.3,lincRNA
61628,11,44116448,-1,FO704721.3,lincRNA


In [21]:

annot.columns=['Chromosome', 'Start', 'Strand', 'Gene', 'Transcript_type']
annot = annot[annot.Transcript_type == 'protein_coding']
annot.Strand[annot.Strand == 1] = '+'
annot.Strand[annot.Strand == -1] = '-'

annot


,Chromosome,Start,Strand,Gene,Transcript_type
0,24,20927202,+,fam162a,protein_coding
1,24,20927135,+,fam162a,protein_coding
2,17,50472985,-,si:ch211-235i11.3,protein_coding
3,17,50472864,-,si:ch211-235i11.3,protein_coding
4,1,49266886,+,caly,protein_coding
...,...,...,...,...,...
60816,MT,11009,+,mt-nd4l,protein_coding
60817,MT,11299,+,mt-nd4,protein_coding
60821,MT,12897,+,mt-nd5,protein_coding
60822,MT,15232,-,mt-nd6,protein_coding


In [25]:


#pr_annot = pr.PyRanges(annot.dropna(axis = 0))


In [22]:
from scenicplus.wrappers.run_pycistarget import run_pycistarget

cistarget_dict=run_pycistarget(region_sets,
                 ctx_db_path = rankings_db,
                 save_path = outDir+'/motifs/',
                 species='custom',
                 custom_annot=annot,
                 dem_db_path = scores_db,
                 run_without_promoters = True,
                 promoter_space = 500,
                 ctx_auc_threshold = 0.005,
                 ctx_nes_threshold = 3.0,
                 ctx_rank_threshold = 0.05,
                 dem_log2fc_thr = 0.5,
                 dem_motif_hit_thr = 3.0,
                 dem_max_bg_regions = 500,
                 path_to_motif_annotations = motif_annotation,
                 annotation_version = 'v10nr_clust',
                 annotation = ['Direct_annot', 'Orthology_annot'],
                 n_cpu = 1,
                  _temp_dir = os.path.join(tmpDir, 'ray_spill'))

2024-04-04 09:59:54,904 pycisTarget_wrapper INFO     Created folder : SCENIC_results_MARCH2024//motifs/
2024-04-04 09:59:54,983 pycisTarget_wrapper INFO     Loading cisTarget database for Topics
2024-04-04 09:59:54,984 cisTarget    INFO     Reading cisTarget database
2024-04-04 10:04:09,010 pycisTarget_wrapper INFO     Running cisTarget for Topics
2024-04-04 10:04:09,012 cisTarget    INFO     Running cisTarget for Topic1 which has 11252 regions
2024-04-04 10:04:21,969 cisTarget    INFO     Annotating motifs for Topic1
2024-04-04 10:04:35,567 cisTarget    INFO     Getting cistromes for Topic1
2024-04-04 10:04:35,620 cisTarget    INFO     Running cisTarget for Topic2 which has 3125 regions
2024-04-04 10:04:41,924 cisTarget    INFO     Annotating motifs for Topic2
2024-04-04 10:04:52,165 cisTarget    INFO     Getting cistromes for Topic2
2024-04-04 10:04:52,684 cisTarget    INFO     Running cisTarget for Topic3 which has 12781 regions
2024-04-04 10:05:05,728 cisTarget    INFO     Annotati

In [24]:
cistarget_dict

In [25]:
import dill

menr = dill.load(open(os.path.join(outDir, 'motifs/menr.pkl'), 'rb'))


In [26]:

menr['DEM_Topics_All'].DEM_results('Topic8')


,Logo,Contrast,Direct_annot,Orthology_annot,Log2FC,Adjusted_pval,Mean_fg,Mean_bg,Motif_hit_thr,Motif_hits
metacluster_91.1,,Topic8,NaN,NaN,2.369932,0.040445,2.04241,0.395114,3.0,279.0
metacluster_86.1,,Topic8,NaN,NaN,2.056939,0.001226,2.524602,0.606726,3.0,402.0
metacluster_160.24,,Topic8,NaN,NaN,1.228117,0.0,4.881072,2.083603,3.0,1141.0
tfdimers__MD00345,,Topic8,"hnf1ba, churc1, hnf1bb","hnf1ba, churc1, hnf1bb",1.181669,0.000161,0.502226,0.221402,3.0,187.0
jaspar__MA1700.1,,Topic8,NaN,NaN,1.091676,0.0,3.131329,1.46927,3.0,946.0
tfdimers__MD00008,,Topic8,ikzf2,ikzf2,1.04456,0.004768,0.49277,0.238891,3.0,183.0
jaspar__MA0543.1,,Topic8,NaN,NaN,1.043336,0.0,4.237372,2.055991,3.0,1328.0
kznf__ZNF398_Imbeault2017_RP_ChIP-seq,,Topic8,NaN,NaN,1.040464,0.0,3.177375,1.544748,3.0,986.0
tfdimers__MD00481,,Topic8,zgc:171929,zgc:171929,1.00068,0.000671,0.539094,0.26942,3.0,213.0
tfdimers__MD00193,,Topic8,"stat1b, stat1a, elf1","stat1b, stat1a, elf1",0.982793,0.002035,0.489076,0.247472,3.0,193.0


: 